# D03 - SQLite with Python

## Understanding Goals
By the end of this task, you should be able to:

- Use a programming language to work with SQL databases
- Use `sqlite3.connect()` to open or create a SQLite file
- Use `sqlite3.Connection.execute()` to run SQL
- Use `sqlite3.Cursor.fetchone()` and `sqlite3.Cursor.fetchall()` to retrieve database rows
- Use `sqlite3.Connection.commit()` to save changes and `sqlite3.Connection.close()` to close SQLite files

Referenced: SQLite Notes by CPDD Computer Education Unit

## Section 1 - Python and SQLite

In the previous lessons, you used DB Browser for SQLite to create SQLite databases and run SQL queries. This is because DB Browser's graphical user interface makes it easy to experiment with SQL and examine the results.

However, DB Browser is not an appropriate program to use if we want to customise or restrict how the contents of a database are modified or presented. Suppose we have a SQLite database that stores information about the books in a library and we want to let users search the database. We should not use DB Browser for this purpose as malicious users can also use DB Browser to run harmful SQL (e.g., DROP TABLE). The interface of DB Browser may also be confusing to users who are not familiar with databases or SQLite. 

Instead, developers typically write custom programs to control how users interact with a database. The program may let the user complete a form or choose from a menu to describe what he/she wants to do. Based on the user's input, the program would then generate the appropriate SQL and run it to produce the intended result. This prevents users from modifying the database in ways that are unexpected to the developer.


### _1.1 Loading an SQLite Database_

To open or create a SQLite database, `import sqlite3` and call `sqlite3.connect()`. This function accepts a str argument that contains the path and filename of an SQLite database file and returns a connection object. If no path is included, the SQLite file is assumed to be in the same directory as the Python file. Furthermore, if the specified file does not exist, an empty file will be created with the given filename instead.

After all operations with the database are complete, the `close()` method of the connection object should then be called. This ensures that the database file is closed properly but does not save any modifications that have been made to the data.

For example, the following Python program tries to load an SQLite database named example.db in the same directory as the program. If such a file does not exist, an empty file named `example.db` will be created instead:


In [58]:
import sqlite3

connection = sqlite3.connect("library.db")
connection.close()

### _1.2 Executing SQL in Python_

After loading an SQLite file and getting a connection, we can execute SQL by calling the connection object's `execute()` method with a str containing the SQL we wish to run. If any data is modified, we can also save our changes by calling the connection object's `commit()` method. For instance, the following Python program creates a new table named `Book` in a new SQLite file named `library.db`.

Upon successful execution, use DB Browser to open `library.db` and verify that the table has been created.

In [59]:
import sqlite3

# init the connection
connection = sqlite3.connect("library.db")

# write the query and execute it
query = """
CREATE TABLE Book (
    ID INTEGER PRIMARY KEY,
    Title TEXT
)
"""
connection.execute(query)

# without commit, the table will not be created
connection.commit()
# remember to close the connection after you are done
connection.close()

However, running the above program again will result in an error as the table `Book` already exists. To prevent this, we can use the `IF NOT EXISTS` clause to check if the table exists before creating it.

Alternatively, we can also use a `try-except` block to catch the exception that is raised when the table already exists.

In [60]:
import sqlite3

# init the connection
connection = sqlite3.connect("library.db")

# write the query and execute it
# Method 1: IF NOT EXISTS (before table name Book)
query = """
CREATE TABLE IF NOT EXISTS Book (
    ID INTEGER PRIMARY KEY,
    Title TEXT
)
"""
connection.execute(query)

# Method 2: try-except
# try:
#     connection.execute(query)
# except sqlite3.OperationalError:
#     print("Table already exists")

# without commit, the table will not be created
connection.commit()
# remember to close the connection after you are done
connection.close()

### _1.3 Committing Changes and Rolling Back_

`commit()` saves all changes that have been made to the database since the last commit. 

However, if we want to undo all changes that have been made since the last commit, we can call the connection object's `rollback()` method instead.

In [61]:
import sqlite3

connection = sqlite3.connect("library.db")

query1 = "INSERT INTO Book(ID, Title) VALUES(1, 'Book 1 - Rollback Book')"
query2 = "INSERT INTO Book(ID, Title) VALUES(2, 'Book 2 - Also Rollback Book')"
connection.execute(query1)
connection.execute(query2)

# rollback the above changes
connection.rollback()

query3 = "INSERT INTO Book(ID, Title) VALUES(3, 'Committed Book')"
connection.execute(query3)

# in the end only query3 will be committed
connection.commit()

connection.close()

### _1.4 Enclosing User Input in SQL Safely_

When generating SQL commands, we often need to include some data that is provided by the user. For instance, we may want the user to enter the ID of a book to delete from the database. This requires us to generate a DELETE command with the entered ID in its WHERE clause.

We may be tempted to use str concatenation in order to generate the required SQL command. Unfortunately, this is insecure as special characters or keywords in the user's input are not escaped and malicious users can take advantage of this to inject his/her own SQL commands.

Instead, we should use parameter substitution to safely include data that is provided by the user. To do this, we use the question-mark character ? as a placeholder in the SQL for any data that is provided by the user. We then provide a second argument to execute() that is a tuple of values that will replace the placeholders. This ensures that the provided values are escaped properly and cannot be misinterpreted as SQL.

For example, the following program safely asks for the ID of a book and deletes the corresponding row from the database:


In [62]:
# if db is locked:
# 1. check if you have opened it using DB Browser
# 2. you can try to run the following to unlock it
# 3. if both methods don't work, restart your computer

import sqlite3
connection = sqlite3.connect("library.db")
connection.close()

In [63]:
import sqlite3

connection = sqlite3.connect("library.db")

# Insert some rows first so we have something to delete
query1 = "INSERT INTO Book(ID, Title) VALUES(4, 'Extra Book')"
connection.execute(query1)

query2 = "INSERT INTO Book(ID, Title) VALUES(5, 'Also Extra Book')"
connection.execute(query2)
connection.commit()

# Ask for ID and delete the corresponding row
book_id = input("Enter Book ID to delete: ")
connection.execute("DELETE FROM Book WHERE ID = ?", (book_id,)) # use this method, not string conc. to prevent sql injection

connection.commit()

connection.close()


## Section 2 - Retrieving Data from SQLite

With the above code structure, we are able to execute most SQL commands in Python. Such as creating tables, inserting data, updating data, and deleting data.

However, there's still one important aspect interacting with SQL database, that is to retrieve data from the database. In this section, we will learn how to retrieve data from SQLite database using Python.

### _2.1 Retrieving rows of data using `fetchall()`_

To retrieve data from a database, we can use the `fetchall()` method of the cursor object returned by `execute()`. This method returns a list of tuples, where each tuple represents a row of data. For instance, the following program retrieves all rows from the `Book` table and prints them out:

In [64]:
# run this cell to reset the Book table

import sqlite3

connection = sqlite3.connect("library.db")

# clear table
connection.execute("DELETE FROM Book")

# Insert a few books into the db
for i in range(10):
    query = "INSERT INTO Book(ID, Title) VALUES(?, ?)"
    connection.execute(query, (i, f"Book {i}"))

connection.commit()

In [66]:
import sqlite3

connection = sqlite3.connect("library.db")

query = "SELECT * FROM Book"

# execute the query and get all the results
cursor = connection.execute(query)
rows = cursor.fetchall()

# close the connection
cursor.close()
connection.close()

# print the results
for row in rows:
    print(row)

(0, 'Book 0')
(1, 'Book 1')
(2, 'Book 2')
(3, 'Book 3')
(4, 'Book 4')
(5, 'Book 5')
(6, 'Book 6')
(7, 'Book 7')
(8, 'Book 8')
(9, 'Book 9')


### _2.2 Retrieving a single row of data using `fetchone()`_

If we only want to retrieve a single row of data, we can use the `fetchone()` method instead. This method returns a tuple that represents the first row of data that is returned by the SQL query. For instance, the following program retrieves the row with ID 1 from the `Book` table.

In [67]:
import sqlite3

connection = sqlite3.connect("library.db")

query = "SELECT * FROM Book WHERE ID = ?"

# execute the query and get all the results
cursor = connection.execute(query, (1,))

# fetchone() returns the first row
# in this case, we are sure that there is only one row
row = cursor.fetchone()

# close the connection
cursor.close()
connection.close()

# print the results
print(row)

(1, 'Book 1')


We can always use `fetchall()[0]` to retrieve the first row of data from a query. However, this is inefficient as the database has to retrieve all rows of data before returning the first row. In contrast, `fetchone()` only retrieves the first row of data and is therefore more efficient.

So in the situation where we only want to retrieve a single row of data, we should use `fetchone()` instead of `fetchall()[0]`.